<a href="https://colab.research.google.com/github/tvani2/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/LightGBMFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/cs231n/assignments/finalproject/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip features.csv.zip
!unzip train.csv.zip
!unzip test.csv.zip
!unzip sampleSubmission.csv.zip

Mounted at /content/drive
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 511MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  features.csv.zip
  inflating: features.csv            
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    


In [ ]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")

In [ ]:
class DataProcessor:
    def merge_data(self, main_df, features_df, stores_df):
        merged = pd.merge(main_df, features_df, on=["Store", "Date", "IsHoliday"], how="left")
        merged = pd.merge(merged, stores_df, on="Store", how="left")
        return merged

    def convert_date(self, df):
        df["Date"] = pd.to_datetime(df["Date"])
        return df

    def extract_date_features(self, df):
        df["Year"] = df["Date"].dt.year
        df["Month"] = df["Date"].dt.month
        df["Week"] = df["Date"].dt.isocalendar().week
        df["Day"] = df["Date"].dt.dayofweek
        return df

    def fill_markdowns(self, df):
        markdown_cols = [f"MarkDown{i}" for i in range(1, 6)]
        df[markdown_cols] = df[markdown_cols].fillna(0)
        return df

    def fill_economics(self, df):
        df["CPI"] = df["CPI"].ffill().bfill()
        df["Unemployment"] = df["Unemployment"].ffill().bfill()
        return df

    def encode_types(self, df):
        df["Type"] = df["Type"].map({"A": 0, "B": 1, "C": 2})
        return df

    def process_data(self, main_df, features_df, stores_df):
        df = self.merge_data(main_df, features_df, stores_df)
        df = self.convert_date(df)
        df = self.extract_date_features(df)
        df = self.fill_markdowns(df)
        df = self.fill_economics(df)
        df = self.encode_types(df)
        return df

In [ ]:
processor = DataProcessor()

train_merged = processor.process_data(train, features, stores)
test_merged = processor.process_data(test, features, stores)

In [ ]:
train_merged.shape, test_merged.shape

((421570, 20), (115064, 19))

In [ ]:
if not train_merged['Date'].is_monotonic_increasing:
    train_merged = train_merged.sort_values(by='Date')

display(train_merged.head())

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Week,Day
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,0,151315,2010,2,5,4
277665,29,5,2010-02-05,15552.08,False,24.36,2.788,0.0,0.0,0.0,0.0,0.0,131.527903,10.064,1,93638,2010,2,5,4
277808,29,6,2010-02-05,3200.22,False,24.36,2.788,0.0,0.0,0.0,0.0,0.0,131.527903,10.064,1,93638,2010,2,5,4
277951,29,7,2010-02-05,10820.05,False,24.36,2.788,0.0,0.0,0.0,0.0,0.0,131.527903,10.064,1,93638,2010,2,5,4
278094,29,8,2010-02-05,20055.64,False,24.36,2.788,0.0,0.0,0.0,0.0,0.0,131.527903,10.064,1,93638,2010,2,5,4


In [ ]:
train_merged['Weekly_Sales_Lag1'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).fillna(0)
train_merged['Weekly_Sales_Lag2'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2).fillna(0)
train_merged['Weekly_Sales_Lag3'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(3).fillna(0)

train_merged['Weekly_Sales_MA4'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(window=4).mean()).fillna(0)
train_merged['Weekly_Sales_MA12'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(window=12).mean()).fillna(0)
train_merged['Weekly_Sales_MA26'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].transform(lambda x: x.rolling(window=26).mean()).fillna(0)

display(train_merged.head())

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Year,Month,Week,Day,Weekly_Sales_Lag1,Weekly_Sales_Lag2,Weekly_Sales_Lag3,Weekly_Sales_MA4,Weekly_Sales_MA12,Weekly_Sales_MA26
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,2010,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0
277665,29,5,2010-02-05,15552.08,False,24.36,2.788,0.0,0.0,0.0,...,2010,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0
277808,29,6,2010-02-05,3200.22,False,24.36,2.788,0.0,0.0,0.0,...,2010,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0
277951,29,7,2010-02-05,10820.05,False,24.36,2.788,0.0,0.0,0.0,...,2010,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0
278094,29,8,2010-02-05,20055.64,False,24.36,2.788,0.0,0.0,0.0,...,2010,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
major_holiday_dates = train_merged[train_merged['IsHoliday'] == True]['Date'].unique()

def days_to_next_holiday(date, holidays):
    future_holidays = holidays[holidays > date]
    if len(future_holidays) > 0:
        return (future_holidays.min() - date).days
    return -1 # Or some other indicator for no future holidays

def days_since_last_holiday(date, holidays):
    past_holidays = holidays[holidays < date]
    if len(past_holidays) > 0:
        return (date - past_holidays.max()).days
    return -1 # Or some other indicator for no past holidays

# Apply holiday proximity features to both train and test data
train_merged['Days_To_Next_Holiday'] = train_merged['Date'].apply(lambda x: days_to_next_holiday(x, major_holiday_dates))
train_merged['Days_Since_Last_Holiday'] = train_merged['Date'].apply(lambda x: days_since_last_holiday(x, major_holiday_dates))

test_merged['Days_To_Next_Holiday'] = test_merged['Date'].apply(lambda x: days_to_next_holiday(x, major_holiday_dates))
test_merged['Days_Since_Last_Holiday'] = test_merged['Date'].apply(lambda x: days_since_last_holiday(x, major_holiday_dates))

holiday_dates = {
    "Super_Bowl": ["2010-02-12", "2011-02-11", "2012-02-10", "2013-02-08"],
    "Labor_Day": ["2010-09-10", "2011-09-09", "2012-09-07", "2013-09-06"],
    "Thanksgiving": ["2010-11-26", "2011-11-25", "2012-11-23", "2013-11-29"],
    "Christmas": ["2010-12-31", "2011-12-30", "2012-12-28", "2013-12-27"]
}

for holiday, dates in holiday_dates.items():
    train_merged[holiday] = train_merged["Date"].isin(pd.to_datetime(dates)).astype(int)
    test_merged[holiday] = test_merged["Date"].isin(pd.to_datetime(dates)).astype(int)

train_merged['IsHoliday'] = train_merged['IsHoliday'].astype(int)
test_merged['IsHoliday'] = test_merged['IsHoliday'].astype(int)

display(train_merged.head())
display(test_merged.head())

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Weekly_Sales_Lag3,Weekly_Sales_MA4,Weekly_Sales_MA12,Weekly_Sales_MA26,Days_To_Next_Holiday,Days_Since_Last_Holiday,Super_Bowl,Labor_Day,Thanksgiving,Christmas
0,1,1,2010-02-05,24924.50,0,42.31,2.572,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,-1,0,0,0,0
277665,29,5,2010-02-05,15552.08,0,24.36,2.788,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,-1,0,0,0,0
277808,29,6,2010-02-05,3200.22,0,24.36,2.788,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,-1,0,0,0,0
277951,29,7,2010-02-05,10820.05,0,24.36,2.788,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,-1,0,0,0,0
278094,29,8,2010-02-05,20055.64,0,24.36,2.788,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7,-1,0,0,0,0


,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Year,Month,Week,Day,Days_To_Next_Holiday,Days_Since_Last_Holiday,Super_Bowl,Labor_Day,Thanksgiving,Christmas
0,1,1,2012-11-02,0,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,2012,11,44,4,-1,56,0,0,0,0
1,1,1,2012-11-09,0,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,2012,11,45,4,-1,63,0,0,0,0
2,1,1,2012-11-16,0,52.92,3.252,9696.28,292.10,103.78,1133.15,...,2012,11,46,4,-1,70,0,0,0,0
3,1,1,2012-11-23,1,56.23,3.211,883.59,4.17,74910.32,209.91,...,2012,11,47,4,-1,77,0,0,1,0
4,1,1,2012-11-30,0,52.34,3.207,2460.03,0.00,3838.35,150.57,...,2012,11,48,4,-1,84,0,0,0,0


In [ ]:
# Determine the split points (e.g., 80% for training, 10% for validation, 10% for testing)
train_size = int(len(train_merged) * 0.8)
val_size = int(len(train_merged) * 0.1)
test_size = len(train_merged) - train_size - val_size

# Find the dates at the calculated split points
train_split_date = train_merged.iloc[train_size]['Date']
val_split_date = train_merged.iloc[train_size + val_size]['Date']

# Find the closest Friday to the split dates within the dataset
# We can iterate through the dates in the merged dataframe to find the exact Friday
def find_closest_friday(date, df_dates):
    closest_friday = None
    for d in df_dates:
        if d >= date and d.dayofweek == 4: # Friday is dayofweek 4
            closest_friday = d
            break
    return closest_friday

all_dates = train_merged['Date'].unique()
train_split_date_friday = find_closest_friday(train_split_date, all_dates)
val_split_date_friday = find_closest_friday(val_split_date, all_dates)

print(f"Original train split date: {train_split_date}")
print(f"Adjusted train split date (Friday): {train_split_date_friday}")
print(f"Original validation split date: {val_split_date}")
print(f"Adjusted validation split date (Friday): {val_split_date_friday}")

# Split the data based on the adjusted Friday dates
train_df = train_merged[train_merged['Date'] < train_split_date_friday]
val_df = train_merged[(train_merged['Date'] >= train_split_date_friday) & (train_merged['Date'] < val_split_date_friday)]
test_df = train_merged[train_merged['Date'] >= val_split_date_friday]

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print("Training set shape after resetting index:", train_df.shape)
print("Validation set shape after resetting index:", val_df.shape)
print("Test set shape after resetting index:", test_df.shape)

Original train split date: 2012-04-13 00:00:00
Adjusted train split date (Friday): 2012-04-13 00:00:00
Original validation split date: 2012-07-20 00:00:00
Adjusted validation split date (Friday): 2012-07-20 00:00:00
Training set shape after resetting index: (335761, 32)
Validation set shape after resetting index: (41394, 32)
Test set shape after resetting index: (44415, 32)


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tvani22 (final-project-ml) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
TARGET = "Weekly_Sales"
DROP_COLS = ["Date", TARGET]

X_train = train_df.drop(columns=DROP_COLS)
X_val = val_df.drop(columns=DROP_COLS)
X_test = test_df.drop(columns=DROP_COLS)

y_train = train_df[TARGET]
y_val = val_df[TARGET]
y_test = test_df[TARGET]

In [ ]:
import numpy as np

def WMAE(y_true, y_pred, weights):
    return np.round(np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights), 5)

train_weights = np.where(train_df["IsHoliday"] == 1, 5, 1)
val_weights = np.where(val_df["IsHoliday"] == 1, 5, 1)
test_weights = np.where(test_df["IsHoliday"] == 1, 5, 1)

In [ ]:
# Check shapes
print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Test shape:", test_df.shape)

# Check date ranges
print("\n--- DATE RANGES ---")
print("Train: ", train_df['Date'].min(), "->", train_df['Date'].max())
print("Val:   ", val_df['Date'].min(), "->", val_df['Date'].max())
print("Test:  ", test_df['Date'].min(), "->", test_df['Date'].max())

# Confirm time order integrity
if train_df['Date'].max() < val_df['Date'].min() < test_df['Date'].min():
    print("\n✅ Date-based split looks consistent (no obvious leakage).")
else:
    print("\n⚠️ WARNING: Date-based splits may be overlapping or unordered.")

# Check for overlapping rows (based on Date + Store + Dept as unique keys)
train_keys = set(zip(train_df['Date'], train_df['Store'], train_df['Dept']))
val_keys = set(zip(val_df['Date'], val_df['Store'], val_df['Dept']))
test_keys = set(zip(test_df['Date'], test_df['Store'], test_df['Dept']))

print("\n--- DUPLICATE CHECKS ---")
print("Train ∩ Val overlap:", len(train_keys & val_keys))
print("Train ∩ Test overlap:", len(train_keys & test_keys))
print("Val ∩ Test overlap:", len(val_keys & test_keys))

if len(train_keys & test_keys) == 0 and len(val_keys & test_keys) == 0:
    print("\n✅ No row leakage between train/val/test.")
else:
    print("\n❌ Data leakage detected between splits!")


Train shape: (335761, 32)
Val shape: (41394, 32)
Test shape: (44415, 32)

--- DATE RANGES ---
Train:  2010-02-05 00:00:00 -> 2012-04-06 00:00:00
Val:    2012-04-13 00:00:00 -> 2012-07-13 00:00:00
Test:   2012-07-20 00:00:00 -> 2012-10-26 00:00:00

✅ Date-based split looks consistent (no obvious leakage).

--- DUPLICATE CHECKS ---
Train ∩ Val overlap: 0
Train ∩ Test overlap: 0
Val ∩ Test overlap: 0

✅ No row leakage between train/val/test.


In [ ]:
X_train.shape, X_val.shape, X_test.shape

((335761, 30), (41394, 30), (44415, 30))

In [ ]:
import lightgbm as lgb
import numpy as np
import wandb

wandb.init(
    project="walmart-forecasting",
    entity="final-project-ml",
    name="lightGBM1",
    config={
        "model": "LightGBM",
        "params": {
            "num_leaves": 70,
            "max_depth": 15,
            "learning_rate": 0.1,
            "min_child_samples": 20,
            "subsample": 0.8,
            "n_estimators": 1000
        },
        "metric": "WMAE"
    }
)

# Prepare model
model = lgb.LGBMRegressor(
    objective="regression",
    num_leaves=70,
    max_depth=15,
    learning_rate=0.1,
    min_child_samples=20,
    subsample=0.8,
    n_estimators=1000,
    random_state=42,
    verbose=-1 # Suppress LightGBM output
)

# Track best score manually
best_val_wmae = float("inf")
early_stopping_rounds = 50
no_improve = 0

for i in range(1, 51):
    model.n_estimators = i
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)

    train_wmae = WMAE(y_train, train_preds, train_weights)
    val_wmae = WMAE(y_val, val_preds, val_weights)

    wandb.log({
        "epoch": i,
        "train_WMAE": train_wmae,
        "val_WMAE": val_wmae
    })

    print(f"Epoch {i}: Train WMAE = {train_wmae:.2f}, Val WMAE = {val_wmae:.2f}")

    if val_wmae < best_val_wmae:
        best_val_wmae = val_wmae
        no_improve = 0
    else:
        no_improve += 1

    if no_improve >= early_stopping_rounds:
        print(f"Early stopping at epoch {i}")
        break

wandb.finish()

epoch,▁▂▃▅▆▇█
train_WMAE,█▆▅▄▃▂▁
val_WMAE,█▇▅▄▃▂▁
epoch,7
train_WMAE,7737.12991
val_WMAE,7443.19162


Epoch 1: Train WMAE = 13904.68, Val WMAE = 13727.58
Epoch 2: Train WMAE = 12570.42, Val WMAE = 12383.81
Epoch 3: Train WMAE = 11382.58, Val WMAE = 11175.04
Epoch 4: Train WMAE = 10316.89, Val WMAE = 10090.68
Epoch 5: Train WMAE = 9362.06, Val WMAE = 9114.21
Epoch 6: Train WMAE = 8505.91, Val WMAE = 8236.79
Epoch 7: Train WMAE = 7737.13, Val WMAE = 7443.19
Epoch 8: Train WMAE = 7047.57, Val WMAE = 6736.73
Epoch 9: Train WMAE = 6425.96, Val WMAE = 6100.87
Epoch 10: Train WMAE = 5866.51, Val WMAE = 5533.35
Epoch 11: Train WMAE = 5371.84, Val WMAE = 5018.94
Epoch 12: Train WMAE = 4928.85, Val WMAE = 4564.49
Epoch 13: Train WMAE = 4534.29, Val WMAE = 4155.98
Epoch 14: Train WMAE = 4178.23, Val WMAE = 3790.81
Epoch 15: Train WMAE = 3863.25, Val WMAE = 3468.67
Epoch 16: Train WMAE = 3581.19, Val WMAE = 3180.51
Epoch 17: Train WMAE = 3330.58, Val WMAE = 2921.86
Epoch 18: Train WMAE = 3107.22, Val WMAE = 2695.53
Epoch 19: Train WMAE = 2908.60, Val WMAE = 2491.75
Epoch 20: Train WMAE = 2736.82, 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_WMAE,█▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_WMAE,█▇▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_WMAE,1367.60498
val_WMAE,1074.63362


# **New Experiment**




In [25]:
# --- YoY Sales Feature (same week, previous year) ---
train_merged['YoY_Sales'] = train_merged.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(52).fillna(0)

# --- Fourier Features for Weekly Seasonality ---
train_merged['sin_week'] = np.sin(2 * np.pi * train_merged['Week'] / 52)
train_merged['cos_week'] = np.cos(2 * np.pi * train_merged['Week'] / 52)

test_merged['sin_week'] = np.sin(2 * np.pi * test_merged['Week'] / 52)
test_merged['cos_week'] = np.cos(2 * np.pi * test_merged['Week'] / 52)

In [26]:
# Determine the split points (e.g., 80% for training, 10% for validation, 10% for testing)
train_size = int(len(train_merged) * 0.8)
val_size = int(len(train_merged) * 0.1)
test_size = len(train_merged) - train_size - val_size

# Find the dates at the calculated split points
train_split_date = train_merged.iloc[train_size]['Date']
val_split_date = train_merged.iloc[train_size + val_size]['Date']

# Find the closest Friday to the split dates within the dataset
# We can iterate through the dates in the merged dataframe to find the exact Friday
def find_closest_friday(date, df_dates):
    closest_friday = None
    for d in df_dates:
        if d >= date and d.dayofweek == 4: # Friday is dayofweek 4
            closest_friday = d
            break
    return closest_friday

all_dates = train_merged['Date'].unique()
train_split_date_friday = find_closest_friday(train_split_date, all_dates)
val_split_date_friday = find_closest_friday(val_split_date, all_dates)

print(f"Original train split date: {train_split_date}")
print(f"Adjusted train split date (Friday): {train_split_date_friday}")
print(f"Original validation split date: {val_split_date}")
print(f"Adjusted validation split date (Friday): {val_split_date_friday}")

# Split the data based on the adjusted Friday dates
train_df = train_merged[train_merged['Date'] < train_split_date_friday]
val_df = train_merged[(train_merged['Date'] >= train_split_date_friday) & (train_merged['Date'] < val_split_date_friday)]
test_df = train_merged[train_merged['Date'] >= val_split_date_friday]

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print("Training set shape after resetting index:", train_df.shape)
print("Validation set shape after resetting index:", val_df.shape)
print("Test set shape after resetting index:", test_df.shape)

Original train split date: 2012-04-13 00:00:00
Adjusted train split date (Friday): 2012-04-13 00:00:00
Original validation split date: 2012-07-20 00:00:00
Adjusted validation split date (Friday): 2012-07-20 00:00:00
Training set shape after resetting index: (335761, 35)
Validation set shape after resetting index: (41394, 35)
Test set shape after resetting index: (44415, 35)


In [27]:
TARGET = "Weekly_Sales"
DROP_COLS = ["Date", TARGET]

X_train = train_df.drop(columns=DROP_COLS)
X_val = val_df.drop(columns=DROP_COLS)
X_test = test_df.drop(columns=DROP_COLS)

y_train = train_df[TARGET]
y_val = val_df[TARGET]
y_test = test_df[TARGET]

In [24]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import itertools
import wandb

# --- Define Hyperparameter Grid ---
param_grid = {
    'num_leaves': [31, 70],
    'max_depth': [10, 15],
    'learning_rate': [0.05],
    'min_child_samples': [20],
    'subsample': [0.8],
    'feature_fraction': [0.8, 0.9, 1.0],
    'bagging_fraction': [0.8, 0.9, 1.0],
    'bagging_freq': [1, 5]
}

# --- Iterate Over All Combinations ---
for combo in itertools.product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), combo))

    # Initialize wandb run
    wandb.init(
        project="walmart-forecasting",
        name=f"lightGBM adv | {params}",
        config=params
    )

    model = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1000,
        random_state=42,
        **params
    )

    best_val_wmae = float("inf")
    early_stopping_rounds = 50
    rounds_no_improve = 0

    for i in range(1, 51):
        model.n_estimators = i
        model.fit(X_train, y_train)

        train_preds = model.predict(X_train)
        val_preds = model.predict(X_val)

        train_wmae = WMAE(y_train, train_preds, train_weights)
        val_wmae = WMAE(y_val, val_preds, val_weights)

        wandb.log({
            "epoch": i,
            "train_WMAE": train_wmae,
            "val_WMAE": val_wmae
        })

        print(f"[{params}] Epoch {i}: Train WMAE = {train_wmae:.2f}, Val WMAE = {val_wmae:.2f}")

        if val_wmae < best_val_wmae:
            best_val_wmae = val_wmae
            rounds_no_improve = 0
        else:
            rounds_no_improve += 1

        if rounds_no_improve >= early_stopping_rounds:
            print(f"Early stopping for {params} at epoch {i}")
            break

    wandb.finish()

epoch,▁▂▃▄▅▅▆▇█
train_WMAE,█▇▆▅▄▃▂▂▁
val_WMAE,█▇▆▅▄▃▂▂▁
epoch,9
train_WMAE,10054.94894
val_WMAE,9710.89563


[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 1: Train WMAE = 14672.21, Val WMAE = 14472.12
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 2: Train WMAE = 13982.61, Val WMAE = 13760.54
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 3: Train WMAE = 13323.21, Val WMAE = 13084.85
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 4: Train WMAE = 12698.48, Val WMAE = 12447.75
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_WMAE,██▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_WMAE,██▇▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_WMAE,2511.52568
val_WMAE,1900.70206


[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 1: Train WMAE = 14672.21, Val WMAE = 14472.12
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 2: Train WMAE = 13983.66, Val WMAE = 13759.73
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 3: Train WMAE = 13324.48, Val WMAE = 13089.02
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 4: Train WMAE = 12698.94, Val WMAE = 12452.20
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_WMAE,██▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_WMAE,█▇▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,50
train_WMAE,2515.52185
val_WMAE,1898.40246


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_WMAE,██▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_WMAE,██▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch,50
train_WMAE,2468.55389
val_WMAE,1867.10872


[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 1: Train WMAE = 14658.38, Val WMAE = 14479.00
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 2: Train WMAE = 13965.27, Val WMAE = 13771.09
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 3: Train WMAE = 13302.66, Val WMAE = 13096.41
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 1}] Epoch 4: Train WMAE = 12675.45, Val WMAE = 12456.67
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_WMAE,██▇▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_WMAE,██▇▇▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_WMAE,2457.09293
val_WMAE,1858.56373


[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 1: Train WMAE = 14658.38, Val WMAE = 14479.00
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 2: Train WMAE = 13961.74, Val WMAE = 13772.27
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 3: Train WMAE = 13299.84, Val WMAE = 13099.68
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.8, 'feature_fraction': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 5}] Epoch 4: Train WMAE = 12671.59, Val WMAE = 12460.10
[{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_samples': 20, 'subsample': 0.

KeyboardInterrupt: 

In [28]:
import lightgbm as lgb
import numpy as np
import wandb

wandb.init(
    project="walmart-forecasting",
    entity="final-project-ml",
    name="lightGBM Last",
    config={
        "model": "LightGBM",
        "params": {
            "num_leaves": 70,
            "max_depth": 15,
            "learning_rate": 0.1,
            "min_child_samples": 20,
            "subsample": 0.8,
            "feature_fraction": 0.9,
            "bagging_fraction": 0.9,
            "bagging_freq": 5,
            "n_estimators": 1000
        },
        "metric": "WMAE"
    }
)

# Prepare model
model = lgb.LGBMRegressor(
    objective="regression",
    num_leaves=70,
    max_depth=15,
    learning_rate=0.1,
    min_child_samples=20,
    subsample=0.8,
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=5,
    n_estimators=1000,
    random_state=42
)

# Track best score manually
best_val_wmae = float("inf")
early_stopping_rounds = 50
no_improve = 0

for i in range(1, 51):
    model.n_estimators = i
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)

    train_wmae = WMAE(y_train, train_preds, train_weights)
    val_wmae = WMAE(y_val, val_preds, val_weights)

    wandb.log({
        "epoch": i,
        "train_WMAE": train_wmae,
        "val_WMAE": val_wmae
    })

    print(f"Epoch {i}: Train WMAE = {train_wmae:.2f}, Val WMAE = {val_wmae:.2f}")

    if val_wmae < best_val_wmae:
        best_val_wmae = val_wmae
        no_improve = 0
    else:
        no_improve += 1

    if no_improve >= early_stopping_rounds:
        print(f"Early stopping at epoch {i}")
        break

wandb.finish()

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train_WMAE,██▇▇▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_WMAE,██▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁
epoch,26
train_WMAE,4868.11249
val_WMAE,4325.10699


Epoch 1: Train WMAE = 13904.13, Val WMAE = 13728.75
Epoch 2: Train WMAE = 12572.83, Val WMAE = 12383.03
Epoch 3: Train WMAE = 11387.31, Val WMAE = 11174.81
Epoch 4: Train WMAE = 10317.00, Val WMAE = 10084.97
Epoch 5: Train WMAE = 9362.62, Val WMAE = 9111.22
Epoch 6: Train WMAE = 8504.55, Val WMAE = 8228.25
Epoch 7: Train WMAE = 7747.89, Val WMAE = 7435.69
Epoch 8: Train WMAE = 7052.26, Val WMAE = 6727.27
Epoch 9: Train WMAE = 6430.14, Val WMAE = 6087.99
Epoch 10: Train WMAE = 5878.61, Val WMAE = 5520.06
Epoch 11: Train WMAE = 5395.82, Val WMAE = 4996.53
Epoch 12: Train WMAE = 4947.51, Val WMAE = 4536.17
Epoch 13: Train WMAE = 4550.49, Val WMAE = 4126.11
Epoch 14: Train WMAE = 4196.10, Val WMAE = 3763.04
Epoch 15: Train WMAE = 3880.93, Val WMAE = 3440.64
Epoch 16: Train WMAE = 3591.48, Val WMAE = 3152.92
Epoch 17: Train WMAE = 3341.09, Val WMAE = 2899.38
Epoch 18: Train WMAE = 3120.48, Val WMAE = 2682.02
Epoch 19: Train WMAE = 2923.50, Val WMAE = 2482.68
Epoch 20: Train WMAE = 2753.97, 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_WMAE,█▇▇▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_WMAE,█▇▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_WMAE,1370.71208
val_WMAE,1081.61612
